# Canu Assembly

Bin's friend is working with MinION and recommended that Canu (a fork of Celera) is the best assembler that he's used. 

https://github.com/marbl/canu

In [ ]:
cd /data1/share/scratch/minion-canu

/opt/bioinformatics-software/minion-software/canu/Linux-amd64/bin/canu \
-p flu-11-9-canu \
-d flu-11-9-canu-out \
genomeSize=32k \
-nanopore-raw flu-11-09.fastq

There weren't any contigs produced, but this did perform some error correction with falcon sense (which is a super pain to install), so I can use those reads to see if it helps with anyhting.

First I'm going to separate the FluB reads out. I assembled the entire 11-9 run which is probably a bad idea. Using the FluDB blast results I will grab the reads that map to FluB and start by assembling the full genome and if that doesn't work then I will separate them out by segment.

# Single genome assembly

WE'll start with the 11-9 FluB genome and see how that goes. Note that the PB1 segment will look wonky due to the lab strain spike in.

In [ ]:
cd /data1/share/scratch/minion-canu/flub-only

grep -B6 'Influenza B' flu-11-9.2d.fludb.xml | grep 'Iteration_query-def' | perl -pe 's/^.+>(.+) FAS.+/$1/g' \
| grep --no-group-separator -A3 -F -f - flu-11-9.2d.fastq  > flu-11-9.2d.flub-reads.fastq

/opt/bioinformatics-software/minion-software/canu/Linux-amd64/bin/canu -p flu-11-9-flub-only-canu \
-d flu-11-9-flub-only-canu-out genomeSize=12k -nanopore-raw flu-11-9.2d.flub-reads.fastq

This resulted in 62 corrected reads and 61 unassembled reads.

It seems like an actual assembly won't be very likely since it will try to stitch together a fragmented genome, though the error correction part may be what we should hone in on. 

Also, they do suggest that we run nanopolish prior to assembling even though they implement falcon-sense so I can give that a shot.

They also provide some documentation on how to assemble low coverage genomes that I will try since this is low coverage, especially after the error correction step (dealing with 61 reads!).
http://canu.readthedocs.org/en/stable/quick-start.html#assembling-low-coverage-datasets

I'll start with the defaults for now.

In [ ]:
cd /data1/share/scratch/minion-canu/flub-only

/opt/bioinformatics-software/minion-software/canu/Linux-amd64/bin/canu -p flu-11-9-flub-only-low-cov-canu \
-d flu-11-9-flub-only-low-cov-canu-out genomeSize=12k corMhapSensitivity=high corMinCoverage=2 errorRate=0.035 \
minOverlapLength=499 corMaxEvidenceErate=0.3 -nanopore-raw flu-11-9.2d.flub-reads.fastq

This seemed to not work as well. There are more error corrected reads which isn't what I expected.

# Single Segment Assembly

Before jumping to combining all FluB runs I want to see if the results differ when trying to assemble one specific segment. We'll start with the HA.

In [ ]:
cd /data1/share/scratch/minion-canu/flub-only

grep -P -B6 'Influenza B.+Segment:4' flu-11-9.2d.fludb.xml | grep 'Iteration_query-def' | \
perl -pe 's/^.+>(.+) FAS.+/$1/g' | grep --no-group-separator -A3 -F -f - flu-11-9.2d.fastq  > flu-11-9.2d.flub-HA-reads.fastq

/opt/bioinformatics-software/minion-software/canu/Linux-amd64/bin/canu -p flu-11-9-flub-HA-only-canu \
-d flu-11-9-flub-HA-only-canu-out genomeSize=12k -nanopore-raw flu-11-9.2d.flub-HA-reads.fastq

This didn't work either. I find it very odd that when trying to assemble the whole FluB genome and the HA segment only that there are the same number of unitigs. This definitely seems like there's some weird partitioning going on here so I need to find out what the parameters are that could be causing this.

# Single Segment Low Coverage Assembly

Canu has parameters to use specifically with low coverage datasets. It basically lowers the amount of overlap and the error stringency so that it yields more reads after error correction. This seemed to work, though it produced two different contigs that are fairly similar. Also, they weren't assembled, which makes sense since these should just be consensus sequences. Not sure why, but mapping the MiSeq data against it should help decide which (if either) is correct.

## HA 

In [ ]:
cd /data1/share/scratch/minion-canu/flub-only

canu -p flu -d flu-11-9-flub-HA-only-low-cov-canu-out genomeSize=2k corMhapSensitivity=high corMinCoverage=2 \
errorRate=0.035 minOverlapLength=499 corMaxEvidenceErate=0.3 -nanopore-raw flu-11-9.2d.flub-HA-reads.fastq

cd flu-11-9-flub-HA-only-low-cov-canu-out/

grep -c '>' *fasta

# align to each other to see how different they are
nucmer --maxmatch flu.unassembled.fasta flu.unassembled.fasta
show-coords -lcT out.delta 

S1 | E1 | S2 | E2 | LEN 1 | LEN 2 | % IDY | LEN R | LEN Q | COV R | COV Q | TAGS
---|----|----|----|-------|-------|-------|-------|-------|-------|-------|-----
1 | 1874 | 1 | 1874 | 1874 | 1874 | 100.00 | 1874 | 1874 | 100.00 | 100.00 | tig00000000 | tig00000000
2 | 1819 | 1874 | 48 | 1818 | 1827 | 97.72 | 1819 | 1874 | 99.95 | 97.49 | tig00000001 | tig00000000
1 | 1819 | 1 | 1819 | 1819 | 1819 | 100.00 | 1819 | 1819 | 100.00 | 100.00 | tig00000001 | tig00000001
48 | 1874 | 1819 | 2 | 1827 | 1818 | 97.72 | 1874 | 1819 | 97.49 | 99.95 | tig00000000 | tig00000001

They're 97.72% identical and they're in opposite orientations, so maybe these are the template an compliment reads? Those should theoretically be merged into one contig..... I guess the best way to assess this would be to assemble other segments to see if this is a trend.

In [ ]:
cd /data1/share/scratch/minion-canu/flub-only/flu-11-9-flub-HA-only-low-cov-canu-out

ln -s ../../../miseq-assembly/flub/flub-11-9-blast-hits-assembly/flub.trimmed.r1.fastq
ln -s ../../../miseq-assembly/flub/flub-11-9-blast-hits-assembly/flub.trimmed.r2.fastq

bwa index flu.unassembled.fasta
bwa mem -t 30 flu.unassembled.fasta flub.trimmed.r1.fastq flub.trimmed.r2.fastq > flub.trimmed.sam && \
samtools view -b -o flub.trimmed.bam flub.trimmed.sam && samtools sort -o flub.trimmed.sort.bam flub.trimmed.bam && \
samtools index flub.trimmed.sort.bam && rm flub.trimmed.sam flub.trimmed.bam

Contig0 definitely seems to be correct, though there are some deletions in the reference that don't agree with the MiSeq data. 
![title](docs/minion-flub-ha-denovo-contig0.png)
![title](docs/minion-flub-ha-denovo-contig0-deletions.png)

Contig1 seems very erroneous. though it seems to have a fair amount of coverage we can't really throw it out as garbage just yet.
![title](docs/minion-flub-ha-denovo-contig1.png)

So before we get ahead of ourselves, let's try to do an assembly with a different segment.

## PB2 

In [ ]:
cd /data1/share/scratch/minion-canu/flub-only

grep -h -P -B6 'Influenza B.+Segment:1' flu-11-9.2d.fludb.xml | grep 'Iteration_query-def' | \
perl -pe 's/^.+>(.+) FAS.+/$1/g' | grep --no-group-separator -A3 -F -f - flu-11-9.2d.fastq  > flu-11-9.2d.flub-PB2-reads.fastq

canu -p flu -d flu-11-9-flub-PB2-only-low-cov-canu-out genomeSize=3k corMhapSensitivity=high corMinCoverage=2 \
errorRate=0.035 minOverlapLength=499 corMaxEvidenceErate=0.3 -nanopore-raw flu-11-9.2d.flub-PB2-reads.fastq

cd flu-11-9-flub-PB2-only-low-cov-canu-out/

grep -c '>' *fasta

# align to each other to see how different they are
nucmer --maxmatch flu.unassembled.fasta flu.unassembled.fasta
show-coords -lcT out.delta 

## MP 

In [ ]:
cd /data1/share/scratch/minion-canu/flub-only

grep -P -B6 'Influenza B.+Segment:7' flu-11-9.2d.fludb.xml | grep 'Iteration_query-def' | \
perl -pe 's/^.+>(.+) FAS.+/$1/g' | grep --no-group-separator -A3 -F -f - flu-11-9.2d.fastq  > flu-11-9.2d.flub-MP-reads.fastq

canu -p flu -d flu-11-9-flub-MP-only-low-cov-canu-out genomeSize=1k corMhapSensitivity=high corMinCoverage=2 \
errorRate=0.1 minOverlapLength=499 corMaxEvidenceErate=0.3 -nanopore-raw flu-11-9.2d.flub-MP-reads.fastq

cd flu-11-9-flub-MP-only-low-cov-canu-out/

grep -c '>' *fasta

# align to each other to see how different they are
nucmer --maxmatch flu.unassembled.fasta flu.unassembled.fasta
show-coords -lcT out.delta 

## PA

In [ ]:
cd /data1/share/scratch/minion-canu/flub-only

seg="PA"
segNum=3
segLen=2.2k

grep -P -B6 'Influenza B.+Segment:'$segNum flu-11-9.2d.fludb.xml | grep 'Iteration_query-def' | \
perl -pe 's/^.+>(.+) FAS.+/$1/g' | grep --no-group-separator -A3 -F -f - flu-11-9.2d.fastq  > flu-11-9.2d.flub-$seg-reads.fastq

canu -p flu -d flu-11-9-flub-$seg-only-low-cov-canu-out genomeSize=$segLen corMhapSensitivity=high corMinCoverage=2 \
errorRate=0.5 minOverlapLength=499 corMaxEvidenceErate=0.4 -nanopore-raw flu-11-9.2d.flub-$seg-reads.fastq

cd flu-11-9-flub-$seg-only-low-cov-canu-out/

grep -c '>' *fasta

# align to each other to see how different they are
nucmer --maxmatch flu.unassembled.fasta flu.unassembled.fasta
show-coords -lcT out.delta 

## NS

In [ ]:
cd /data1/share/scratch/minion-canu/flub-only

grep -P -B6 'Influenza B.+Segment:1' flu-11-9.2d.fludb.xml | grep 'Iteration_query-def' | \
perl -pe 's/^.+>(.+) FAS.+/$1/g' | grep --no-group-separator -A3 -F -f - flu-11-9.2d.fastq  > flu-11-9.2d.flub-PB2-reads.fastq

canu -p flu -d flu-11-9-flub-PB2-only-low-cov-canu-out genomeSize=3k corMhapSensitivity=high corMinCoverage=2 \
errorRate=0.035 minOverlapLength=499 corMaxEvidenceErate=0.3 -nanopore-raw flu-11-9.2d.flub-PB2-reads.fastq

cd flu-11-9-flub-PB2-only-low-cov-canu-out/

grep -c '>' *fasta

# align to each other to see how different they are
nucmer --maxmatch flu.unassembled.fasta flu.unassembled.fasta
show-coords -lcT out.delta 

# All FluB Runs

Each segment has produced at least two contigs when assembling. This is workable if you have some way of validating which contig is correct (MiSeq data). However, for clinical purposes this isn't usable and I'm assuming it's a depth thing since the error I'm receiving is telling me that many of the reads are being thrown out. With that said, combining the runs may provide enough data to assembly into one contig.

## HA

This produced two contigs so may be a good start since the data quality seems reasonable.

In [ ]:
cd /data1/share/scratch/minion-canu/flub-only

ln -s ~/projects/MinION-notebook/raw-data/MinION/8-13/flu-8-13.2d.fastq
ln -s ~/projects/MinION-notebook/raw-data/MinION/6-4/flu-6-4.2d.fastq
ln -s ~/projects/MinION-notebook/minion-blast/8-13/*xml
ln -s ~/projects/MinION-notebook/minion-blast/6-4/*xml

grep -h -P -B6 'Influenza B.+Segment:4' *xml | grep 'Iteration_query-def' | perl -pe 's/^.+>(.+) FAS.+/$1/g' \
| grep -h --no-group-separator -A3 -F -f - *2d.fastq > all-flub-HA.2d.fastq

canu -p flu -d all-flub-HA-only-low-cov-canu-out genomeSize=2k corMhapSensitivity=high corMinCoverage=2 errorRate=0.035 \
minOverlapLength=499 corMaxEvidenceErate=0.3 cormhapMemory=100 obtmhapMemory=100 utgmhapMemory=100 \
-nanopore-raw all-flub-HA.2d.fastq

cd all-flub-HA-only-low-cov-canu-out/

grep -c '>' *fasta

# align to each other to see how different they are
nucmer --maxmatch flu.unassembled.fasta flu.unassembled.fasta
show-coords -lcT out.delta 

There seems to be a sweet spot for the error rate, and 0.035 seems to be it (which is way more stringent that I was expecting). Almost every other value produced 29 contigs, while one produced 15 instead of the 12 that we're getting with the settings above. I'm assuming there will need to be a sweep in these values for each segment since it varies in the single run analysis. Subsequently I will need to resequence these to check which has the best coverage since the error correction clusters the raw reads into a representative so the coverage in the contigs files isn't accurate.

So now to run this on all segments. Note that I didn't test for the best parameter settings. I don't think that the number of contigs is something to focus on given the error rate but I'm not sure.

In [ ]:
cd /home/alan/projects/MinION-notebook/minion-canu/flub-only

cat flu-11-9.2d.fastq flu-6-4.2d.fastq flu-8-13.2d.fastq > all-flub-runs.fastq

seg='PB2'
segNum='1'
segSize='2.3k'

grep -h -P -B6 'Influenza B.+Segment:'$segNum *xml | grep 'Iteration_query-def' | \
perl -pe 's/^.+>(.+) FAS.+/$1/g' | grep --no-group-separator -A3 -F -f - all-flub-runs.fastq  > all-flub-runs-$seg.fastq

canu maxThreads=30 maxMemory=100 stopOnReadQuality=false -p flu -d all-flub-runs-$seg-canu-out \
genomeSize=$segSize corMhapSensitivity=high corMinCoverage=2 errorRate=0.035 minOverlapLength=1499 \
minReadLength=1500 corMaxEvidenceErate=0.3 -nanopore-raw all-flub-runs-$seg.fastq

seg='PB1'
segNum='2'
segSize='2.3k'

grep -h -P -B6 'Influenza B.+Segment:'$segNum *xml | grep 'Iteration_query-def' | \
perl -pe 's/^.+>(.+) FAS.+/$1/g' | grep --no-group-separator -A3 -F -f - all-flub-runs.fastq  > all-flub-runs-$seg.fastq

canu maxThreads=30 maxMemory=100 stopOnReadQuality=false -p flu -d all-flub-runs-$seg-canu-out \
genomeSize=$segSize corMhapSensitivity=high corMinCoverage=2 errorRate=0.035 minOverlapLength=1499 \
minReadLength=1500 corMaxEvidenceErate=0.3 -nanopore-raw all-flub-runs-$seg.fastq

seg='PA'
segNum='3'
segSize='2.2k'

grep -h -P -B6 'Influenza B.+Segment:'$segNum *xml | grep 'Iteration_query-def' | \
perl -pe 's/^.+>(.+) FAS.+/$1/g' | grep --no-group-separator -A3 -F -f - all-flub-runs.fastq  > all-flub-runs-$seg.fastq

canu maxThreads=30 maxMemory=100 stopOnReadQuality=false -p flu -d all-flub-runs-$seg-canu-out \
genomeSize=$segSize corMhapSensitivity=high corMinCoverage=2 errorRate=0.035 minOverlapLength=1499 \
minReadLength=1500 corMaxEvidenceErate=0.3 -nanopore-raw all-flub-runs-$seg.fastq

seg='HA'
segNum='4'
segSize='1.7k'

grep -h -P -B6 'Influenza B.+Segment:'$segNum *xml | grep 'Iteration_query-def' | \
perl -pe 's/^.+>(.+) FAS.+/$1/g' | grep --no-group-separator -A3 -F -f - all-flub-runs.fastq  > all-flub-runs-$seg.fastq

canu maxThreads=30 maxMemory=100 stopOnReadQuality=false -p flu -d all-flub-runs-$seg-canu-out \
genomeSize=$segSize corMhapSensitivity=high corMinCoverage=2 errorRate=0.035 minOverlapLength=1499 \
minReadLength=1500 corMaxEvidenceErate=0.3 -nanopore-raw all-flub-runs-$seg.fastq

seg='NP'
segNum='5'
segSize='1.5k'

grep -h -P -B6 'Influenza B.+Segment:'$segNum *xml | grep 'Iteration_query-def' | \
perl -pe 's/^.+>(.+) FAS.+/$1/g' | grep --no-group-separator -A3 -F -f - all-flub-runs.fastq  > all-flub-runs-$seg.fastq

canu maxThreads=30 maxMemory=100 stopOnReadQuality=false -p flu -d all-flub-runs-$seg-canu-out \
genomeSize=$segSize corMhapSensitivity=high corMinCoverage=2 errorRate=0.035 minOverlapLength=1499 \
minReadLength=1500 corMaxEvidenceErate=0.3 -nanopore-raw all-flub-runs-$seg.fastq

seg='NA'
segNum='6'
segSize='1.4k'

grep -h -P -B6 'Influenza B.+Segment:'$segNum *xml | grep 'Iteration_query-def' | \
perl -pe 's/^.+>(.+) FAS.+/$1/g' | grep --no-group-separator -A3 -F -f - all-flub-runs.fastq  > all-flub-runs-$seg.fastq

canu maxThreads=30 maxMemory=100 stopOnReadQuality=false -p flu -d all-flub-runs-$seg-canu-out \
genomeSize=$segSize corMhapSensitivity=high corMinCoverage=2 minOverlapLength=499 \
-nanopore-raw all-flub-runs-$seg.fastq

seg='MP'
segNum='7'
segSize='1k'

grep -h -P -B6 'Influenza B.+Segment:'$segNum *xml | grep 'Iteration_query-def' | \
perl -pe 's/^.+>(.+) FAS.+/$1/g' | grep --no-group-separator -A3 -F -f - all-flub-runs.fastq  > all-flub-runs-$seg.fastq

canu maxThreads=30 maxMemory=100 stopOnReadQuality=false -p flu -d all-flub-runs-$seg-canu-out \
genomeSize=$segSize corMhapSensitivity=high corMinCoverage=2 minOverlapLength=499 \
-nanopore-raw all-flub-runs-$seg.fastq

seg='NS'
segNum='8'
segSize='900'

grep -h -P -B6 'Influenza B.+Segment:'$segNum *xml | grep 'Iteration_query-def' | \
perl -pe 's/^.+>(.+) FAS.+/$1/g' | grep --no-group-separator -A3 -F -f - all-flub-runs.fastq  > all-flub-runs-$seg.fastq

canu maxThreads=30 maxMemory=100 stopOnReadQuality=false -p flu -d all-flub-runs-$seg-canu-out \
genomeSize=$segSize corMhapSensitivity=high corMinCoverage=2 minOverlapLength=499 \
-nanopore-raw all-flub-runs-$seg.fastq

for i in HA NA NP NS MP PA PB1 PB2; do perl -pe 's/^>(tig\d+)/>'$i'-$1/g' all-flub-runs-HA-canu-out/flu.unassembled.fasta; done \
> all-flub-runs.contigs.fasta

I'm curious as to why there are so many contigs and how similar they are to each other. As I said, I'm reluctant to shoot for the fewest contigs given the error rate (if it HAS to incorporate very erroneous reads then it could be distorting the base composition even more) but the fact that there are this many seems strange.

We can start with the HA since it has 25 contigs.

In [ ]:
cd /home/alan/projects/MinION-notebook/minion-canu/flub-only/all-flub-runs-HA-canu-out

nucmer --maxmatch flu.unassembled.fasta flu.unassembled.fasta
show-coords -lcTH out.delta | cut -f7 > pctids.txt

So they're all ~98% similar. Let's see what the MiSeq data have to say about this.

![minion flub assembly](docs/minion-flub-ha-assembled-pctid.png)

In [ ]:
cd ~/projects/MinION-notebook/minion-canu/flub-only/all-flub-runs-HA-canu-out

ln -s ~/projects/MinION-notebook/raw-data/MiSeq/clinical-strains/f3.r1.fastq.gz
ln -s ~/projects/MinION-notebook/raw-data/MiSeq/clinical-strains/f3.r2.fastq.gz

bwa index flu.unassembled.fasta
bwa mem -t 30 -x intractg flu.unassembled.fasta f3.r1.fastq.gz f3.r2.fastq.gz > f3.sam
samtools view -b -o f3.bam f3.sam && samtools sort -o f3.sort.bam f3.bam && samtools index f3.sort.bam
rm f3.bam f3.sam

So this was horrible. Lots of reads mapped to each contig but the coverage profiles for each were completely scattered - not a single contig was completely covered. On top of that, these were very poorly mapped reads. Here's an example:

![miseq to minion flub ha contig](docs/minoin-flub-ha-miseq-alignment.png)

So that was one of the worst case scenarios. However, when compared to the MiSeq consensus sequences they are ~87% similar. 

S1 | E1 | S2 | E2 | LEN 1 | LEN 2 | % IDY | LEN R | LEN Q | COV R | COV Q | TAGS
---|----|----|----|-------|-------|-------|-------|-------|-------|-------|-----
401 | 2249 | 6 | 1902 | 1849 | 1897 | 87.46 | 2424 | 1905 | 76.28 | 99.58 | tig00000007 | NODE_1_length_1905_cov_52.721_ID_5
509 | 2353 | 6 | 1902 | 1845 | 1897 | 87.25 | 2493 | 1905 | 74.01 | 99.58 | tig00000009 | NODE_1_length_1905_cov_52.721_ID_5
156 | 1990 | 6 | 1902 | 1835 | 1897 | 86.62 | 2157 | 1905 | 85.07 | 99.58 | tig00000012 | NODE_1_length_1905_cov_52.721_ID_5
157 | 1962 | 6 | 1902 | 1806 | 1897 | 85.57 | 2129 | 1905 | 84.83 | 99.58 | tig00000004 | NODE_1_length_1905_cov_52.721_ID_5
355 | 2199 | 6 | 1902 | 1845 | 1897 | 87.20 | 2279 | 1905 | 80.96 | 99.58 | tig00000015 | NODE_1_length_1905_cov_52.721_ID_5
408 | 2246 | 12 | 1902 | 1839 | 1891 | 87.16 | 2406 | 1905 | 76.43 | 99.27 | tig00000006 | NODE_1_length_1905_cov_52.721_ID_5
212 | 2063 | 1902 | 6 | 1852 | 1897 | 87.57 | 2690 | 1905 | 68.85 | 99.58 | tig00000000 | NODE_1_length_1905_cov_52.721_ID_5
87 | 1942 | 1902 | 6 | 1856 | 1897 | 87.62 | 2598 | 1905 | 71.44 | 99.58 | tig00000001 | NODE_1_length_1905_cov_52.721_ID_5
78 | 1934 | 1902 | 6 | 1857 | 1897 | 87.88 | 2601 | 1905 | 71.40 | 99.58 | tig00000002 | NODE_1_length_1905_cov_52.721_ID_5
188 | 2033 | 1902 | 6 | 1846 | 1897 | 87.30 | 2668 | 1905 | 69.19 | 99.58 | tig00000003 | NODE_1_length_1905_cov_52.721_ID_5
150 | 1988 | 1902 | 6 | 1839 | 1897 | 86.59 | 2426 | 1905 | 75.80 | 99.58 | tig00000005 | NODE_1_length_1905_cov_52.721_ID_5
191 | 2049 | 1902 | 6 | 1859 | 1897 | 87.93 | 2712 | 1905 | 68.55 | 99.58 | tig00000014 | NODE_1_length_1905_cov_52.721_ID_5
191 | 2048 | 1902 | 6 | 1858 | 1897 | 87.78 | 2710 | 1905 | 68.56 | 99.58 | tig00000008 | NODE_1_length_1905_cov_52.721_ID_5
20 | 1872 | 1902 | 6 | 1853 | 1897 | 87.36 | 2535 | 1905 | 73.10 | 99.58 | tig00000010 | NODE_1_length_1905_cov_52.721_ID_5
181 | 2036 | 1902 | 6 | 1856 | 1897 | 87.67 | 2702 | 1905 | 68.69 | 99.58 | tig00000011 | NODE_1_length_1905_cov_52.721_ID_5
7 | 1842 | 1875 | 6 | 1836 | 1870 | 87.77 | 2460 | 1905 | 74.63 | 98.16 | tig00000013 | NODE_1_length_1905_cov_52.721_ID_5


Let's try with something that has fewer contigs. This should be enough evidence to see if fewer is actually better in this case. Let's try the NP segment (this, as well as the PA and NP segments) have three contigs.

In [ ]:
cd ~/projects/MinION-notebook/minion-canu/flub-only/all-flub-runs-NP-canu-out

ln -s ~/projects/MinION-notebook/raw-data/MiSeq/clinical-strains/f3.r1.fastq.gz
ln -s ~/projects/MinION-notebook/raw-data/MiSeq/clinical-strains/f3.r2.fastq.gz

bwa index flu.unassembled.fasta
bwa mem -t 30 -x intractg flu.unassembled.fasta f3.r1.fastq.gz f3.r2.fastq.gz > f3.sam
samtools view -b -o f3.bam f3.sam && samtools sort -o f3.sort.bam f3.bam && samtools index f3.sort.bam
rm f3.bam f3.sam

So this one has a contig that is fully covered, though low coverage.

![minion flub np contig miseq alignment](docs/minion-flub-np-miseq-alignment.png)

So now that we know it's something close to what we sequenced, we can compare the MiSeq contigs to it, to get a ballpark. There is a chance that this contig is something that was underrepresented in the MiSeq data, but that seems unlikely.

In [ ]:
cd ~/projects/MinION-notebook/minion-canu/flub-only/all-flub-runs-NP-canu-out

nucmer --maxmatch flu.unassembled.fasta \
~/projects/MinION-notebook/miseq-assembly/flub/flub-11-9-blast-hits-assembly/fluB.11-9-top-seg-hits.contigs.fasta

show-coords -lcT out.delta | perl -pe 's/\t/ | /g'

98% similar to the MiSeq consensus. Not bad but also not sure if shooting for fewer contigs would be worthwhile.

S1 | E1 | S2 | E2 | LEN 1 | LEN 2 | % IDY | LEN R | LEN Q | COV R | COV Q | TAGS
---|----|----|----|-------|-------|-------|-------|-------|-------|-------|-----
17 | 1810 | 1 | 1844 | 1794 | 1844 | 97.23 | 1815 | 1861 | 98.84 | 99.09 | tig00000001 | NODE_1_length_1861_cov_52.0934_ID_5
17 | 1759 | 1 | 1790 | 1743 | 1790 | 97.32 | 1759 | 1861 | 99.09 | 96.18 | tig00000002 | NODE_1_length_1861_cov_52.0934_ID_5
4 | 1828 | 1860 | 1 | 1825 | 1860 | 98.01 | 1836 | 1861 | 99.40 | 99.95 | tig00000000 | NODE_1_length_1861_cov_52.0934_ID_5

# All H1N1 Runs

In [ ]:
cd /home/alan/projects/MinION-notebook/minion-canu/h1n1

ln -s ~/projects/MinION-notebook/raw-data/MinION/8-13/flu-8-13.2d.fastq
ln -s ~/projects/MinION-notebook/raw-data/MinION/6-4/flu-6-4.2d.fastq

cat flu-* > all-h1n1-runs.fastq

ln -s ~/projects/MinION-notebook/minion-blast/6-4/flu-6-4.2d.fludb.xml
ln -s ~/projects/MinION-notebook/minion-blast/8-13/flu-8-13.2d.fludb.xml

seg='PB2'
segNum='1'
segSize='2.3k'
subtype='h1n1'

grep -h -P -B6 'Influenza A.+Segment:'$segNum'\|Subtype:H1N1' *xml | grep 'Iteration_query-def' | \
perl -pe 's/^.+>(.+) FAS.+/$1/g' | grep --no-group-separator -A3 -F -f - all-$subtype-runs.fastq  > all-$subtype-runs-$seg.fastq

canu maxThreads=30 maxMemory=100 stopOnReadQuality=false -p flu -d all-$subtype-runs-$seg-canu-out \
genomeSize=$segSize corMhapSensitivity=high corMinCoverage=2 errorRate=0.035 minOverlapLength=1499 \
minReadLength=1500 corMaxEvidenceErate=0.3 -nanopore-raw all-$subtype-runs-$seg.fastq

seg='PB1'
segNum='2'
segSize='2.3k'

grep -h -P -B6 'Influenza A.+Segment:'$segNum'\|Subtype:H1N1' *xml | grep 'Iteration_query-def' | \
perl -pe 's/^.+>(.+) FAS.+/$1/g' | grep --no-group-separator -A3 -F -f - all-$subtype-runs.fastq  > all-$subtype-runs-$seg.fastq

canu maxThreads=30 maxMemory=100 stopOnReadQuality=false -p flu -d all-$subtype-runs-$seg-canu-out \
genomeSize=$segSize corMhapSensitivity=high corMinCoverage=2 errorRate=0.035 minOverlapLength=1499 \
minReadLength=1500 corMaxEvidenceErate=0.3 -nanopore-raw all-$subtype-runs-$seg.fastq

seg='PA'
segNum='3'
segSize='2.2k'

grep -h -P -B6 'Influenza A.+Segment:'$segNum'\|Subtype:H1N1' *xml | grep 'Iteration_query-def' | \
perl -pe 's/^.+>(.+) FAS.+/$1/g' | grep --no-group-separator -A3 -F -f - all-$subtype-runs.fastq  > all-$subtype-runs-$seg.fastq

canu maxThreads=30 maxMemory=100 stopOnReadQuality=false -p flu -d all-$subtype-runs-$seg-canu-out \
genomeSize=$segSize corMhapSensitivity=high corMinCoverage=2 errorRate=0.035 minOverlapLength=1499 \
minReadLength=1500 corMaxEvidenceErate=0.3 -nanopore-raw all-$subtype-runs-$seg.fastq

seg='HA'
segNum='4'
segSize='1.7k'

grep -h -P -B6 'Influenza A.+Segment:'$segNum'\|Subtype:H1N1' *xml | grep 'Iteration_query-def' | \
perl -pe 's/^.+>(.+) FAS.+/$1/g' | grep --no-group-separator -A3 -F -f - all-$subtype-runs.fastq  > all-$subtype-runs-$seg.fastq

canu maxThreads=30 maxMemory=100 stopOnReadQuality=false -p flu -d all-$subtype-runs-$seg-canu-out \
genomeSize=$segSize corMhapSensitivity=high corMinCoverage=2 errorRate=0.035 minOverlapLength=1499 \
minReadLength=1500 corMaxEvidenceErate=0.3 -nanopore-raw all-$subtype-runs-$seg.fastq

seg='NP'
segNum='5'
segSize='1.5k'

grep -h -P -B6 'Influenza A.+Segment:'$segNum'\|Subtype:H1N1' *xml | grep 'Iteration_query-def' | \
perl -pe 's/^.+>(.+) FAS.+/$1/g' | grep --no-group-separator -A3 -F -f - all-$subtype-runs.fastq  > all-$subtype-runs-$seg.fastq

canu maxThreads=30 maxMemory=100 stopOnReadQuality=false -p flu -d all-$subtype-runs-$seg-canu-out \
genomeSize=$segSize corMhapSensitivity=high corMinCoverage=2 errorRate=0.035 minOverlapLength=1499 \
minReadLength=1500 corMaxEvidenceErate=0.3 -nanopore-raw all-$subtype-runs-$seg.fastq

seg='NA'
segNum='6'
segSize='1.4k'

grep -h -P -B6 'Influenza A.+Segment:'$segNum'\|Subtype:H1N1' *xml | grep 'Iteration_query-def' | \
perl -pe 's/^.+>(.+) FAS.+/$1/g' | grep --no-group-separator -A3 -F -f - all-$subtype-runs.fastq  > all-$subtype-runs-$seg.fastq

canu maxThreads=30 maxMemory=100 stopOnReadQuality=false -p flu -d all-$subtype-runs-$seg-canu-out \
genomeSize=$segSize corMhapSensitivity=high corMinCoverage=2 minOverlapLength=499 \
-nanopore-raw all-$subtype-runs-$seg.fastq

seg='MP'
segNum='7'
segSize='1k'

grep -h -P -B6 'Influenza A.+Segment:'$segNum'\|Subtype:H1N1' *xml | grep 'Iteration_query-def' | \
perl -pe 's/^.+>(.+) FAS.+/$1/g' | grep --no-group-separator -A3 -F -f - all-$subtype-runs.fastq  > all-$subtype-runs-$seg.fastq

canu maxThreads=30 maxMemory=100 stopOnReadQuality=false -p flu -d all-$subtype-runs-$seg-canu-out \
genomeSize=$segSize corMhapSensitivity=high corMinCoverage=2 minOverlapLength=499 \
-nanopore-raw all-$subtype-runs-$seg.fastq

seg='NS'
segNum='8'
segSize='900'

grep -h -P -B6 'Influenza A.+Segment:'$segNum'\|Subtype:H1N1' *xml | grep 'Iteration_query-def' | \
perl -pe 's/^.+>(.+) FAS.+/$1/g' | grep --no-group-separator -A3 -F -f - all-$subtype-runs.fastq  > all-$subtype-runs-$seg.fastq

canu maxThreads=30 maxMemory=100 stopOnReadQuality=false -p flu -d all-$subtype-runs-$seg-canu-out \
genomeSize=$segSize minReadLength=500 corMhapSensitivity=high corMinCoverage=2 minOverlapLength=499 \
-nanopore-raw all-$subtype-runs-$seg.fastq

S1 | E1 | S2 | E2 | LEN 1 | LEN 2 | % IDY | LEN R | LEN Q | COV R | COV Q | TAGS
---|----|----|----|-------|-------|-------|-------|-------|-------|-------|-----
36 | 1797 | 1708 | 19 | 1762 | 1690 | 95.86 | 1803 | 1708 | 97.73 | 98.95 | HA | HA-tig00000000
16 | 1048 | 1028 | 14 | 1033 | 1015 | 98.16 | 1048 | 1029 | 98.57 | 98.64 | MP | MP-tig00000000
1 | 1472 | 1462 | 16 | 1472 | 1447 | 98.23 | 1478 | 1483 | 99.59 | 97.57 | NA | NA-tig00000000
1 | 1576 | 18 | 1569 | 1576 | 1552 | 98.16 | 1582 | 1569 | 99.62 | 98.92 | NP | NP-tig00000005
17 | 903 | 872 | 1 | 887 | 872 | 98.09 | 908 | 900 | 97.69 | 96.89 | NS | NS-tig00000000
1 | 2237 | 17 | 2220 | 2237 | 2204 | 98.48 | 2253 | 2220 | 99.29 | 99.28 | PA | PA-tig00000000
1 | 2242 | 3 | 2204 | 2242 | 2202 | 97.99 | 2253 | 2204 | 99.51 | 99.91 | PA | PA-tig00000005
13 | 2363 | 2258 | 3 | 2351 | 2256 | 95.79 | 2370 | 2258 | 99.20 | 99.91 | PB1 | PB1-tig00000000
5 | 2351 | 13 | 2293 | 2347 | 2281 | 97.06 | 2367 | 2294 | 99.16 | 99.43 | PB2 | PB2-tig00000000
164 | 2354 | 2124 | 1 | 2191 | 2124 | 96.90 | 2367 | 2124 | 92.56 | 100.00 | PB2 | PB2-tig00000005



# Canu PacBio Lab Strain Assembly

One thing that Bin suggested is to try to assembly the PacBio lab strain data to see if this is something that is even possible, and if so what's the error rate look like. This will be a good assessment of Canu as a tool as well as the feasiblity of assembling noisier data.

Just to get a feel for how Canu works on cleaner data, I tried assembling all the lab strains together. Didn't work out so well.

In [ ]:
cd ~/projects/MinION-notebook/pacbio-canu

canu minReadLength=500 maxThreads=30 maxMemory=100 -p flu -d pacbio-lab-strains genomeSize=36k \
-pacbio-raw Elodie_sidebar_reads_of_insert.fastq

So the next step is to split them up into segments as I have done before

In [ ]:
cd ~/projects/MinION-notebook/pacbio-canu

for i in $(samtools idxstats pacbio-quads.sort.bam | cut -f1 | grep -v '*'); do samtools view -h pacbio-quads.sort.bam $i\
| samtools view -b -o $i.bam && bamtools convert -in $i.bam -format fastq > $i.fastq && rm $i.bam ; done

#for i in $(samtools idxstats pacbio-quads.sort.bam | cut -f1 | grep -v '*'); do canu minReadLength=500 maxThreads=30 \
#maxMemory=100 -p flu -d $i-canu-out genomeSize=36k -pacbio-raw $i.fastq; done

for i in $(samtools idxstats pacbio-quads.sort.bam | cut -f1 | grep -v '*'); do canu minReadLength=500 maxThreads=30 \
maxMemory=100 -p flu -d $i-canu-out genomeSize=1500 -pacbio-raw $i.fastq; done

So this isn't that bad but there definitely seems to be room to improve.

Segment | Contigs
--------|--------
FluB-HA | 24
FluB-MP | 3
FluB-NA | 5
FluB-NP | 31
FluB-NS | 17
FluB-PA | 26
FluB-PB1 | 26
FluB-PB2 | 25
H1N1-HA | 32
H1N1-MP | 18
H1N1-NA | 29
H1N1-NP | 8
H1N1-NS | 14
H1N1-PA | 21
H1N1-PB1 | 12
H1N1-PB2 | 26
H3N2-HA | 30
H3N2-MP | 15
H3N2-NA | 36
H3N2-NP | 30
H3N2-NS | 11
H3N2-PA | 27
H3N2-PB1 | 26
H3N2-PB2 | 17

FluB-NA has few contigs, so presumably the best it can get with these settings. So, compared to the MiSeq, they are pretty similar.

S1 | E1 | S2 | E2 | LEN 1 | LEN 2 | % IDY | LEN R | LEN Q | COV R | COV Q | TAGS
---|----|----|----|-------|-------|-------|-------|-------|-------|-------|-----
1 | 1553 | 102 | 1654 | 1553 | 1553 | 99.94 | 1553 | 1676 | 100.00 | 92.66 | tig00000003 | NODE_1_length_1676_cov_52.7844_ID_5
1 | 1556 | 103 | 1658 | 1556 | 1556 | 99.94 | 1556 | 1676 | 100.00 | 92.84 | tig00000004 | NODE_1_length_1676_cov_52.7844_ID_5
12 | 1580 | 1674 | 102 | 1569 | 1573 | 99.68 | 1580 | 1676 | 99.30 | 93.85 | tig00000000 | NODE_1_length_1676_cov_52.7844_ID_5
21 | 1564 | 1673 | 113 | 1544 | 1561 | 98.85 | 1564 | 1676 | 98.72 | 93.14 | tig00000001 | NODE_1_length_1676_cov_52.7844_ID_5
21 | 1576 | 1674 | 114 | 1556 | 1561 | 99.62 | 1576 | 1676 | 98.73 | 93.14 | tig00000002 | NODE_1_length_1676_cov_52.7844_ID_5

So if these errors are near the termini then that would be great. If not, then maybe this isn't feasible. 

In [ ]:
cd /home/alan/projects/MinION-notebook/pacbio-canu

ln -s ~/projects/MinION-notebook/miseq-assembly/lab-strains/all-miseq-assemblies.fasta

for i in $(samtools idxstats pacbio-quads.sort.bam | cut -f1 | grep -v '*'); do cat $i-canu-out/flu.unassembled.fasta \
| perl -pe 's/^>/>'$i'-/g'; done > all-pacbio-assemblies.fasta

nucmer all-miseq-assemblies.fasta all-pacbio-assemblies.fasta
show-coords -lcTH <(delta-filter -l 500 -1 -o 10  out.delta ) > out.filter

S1 | E1 | S2 | E2 | LEN 1 | LEN 2 | % IDY | LEN R | LEN Q | COV R | COV Q | TAGS
---|----|----|----|-------|-------|-------|-------|-------|-------|-------|-----
8 | 1906 | 1 | 1897 | 1899 | 1897 | 99.79 | 1906 | 2704 | 99.63 | 70.16 | FluB-HA-NODE_1_length_1906_cov_50.5003_ID_5 | FluB-HA-tig00000018
17 | 1211 | 1195 | 1 | 1195 | 1195 | 99.92 | 1217 | 1195 | 98.19 | 100.00 | FluB-MP-NODE_1_length_1217_cov_56.7908_ID_5 | FluB-MP-tig00000000
102 | 1674 | 1580 | 12 | 1573 | 1569 | 99.68 | 1676 | 1580 | 93.85 | 99.30 | FluB-NA-NODE_1_length_1676_cov_52.7844_ID_5 | FluB-NA-tig00000000
1 | 1726 | 1725 | 1 | 1726 | 1725 | 99.94 | 1988 | 1847 | 86.82 | 93.39 | FluB-NP-NODE_1_length_1988_cov_51.8023_ID_3 | FluB-NP-tig00000003
125 | 1262 | 1135 | 1 | 1138 | 1135 | 99.65 | 1269 | 1980 | 89.68 | 57.32 | FluB-NS-NODE_1_length_1269_cov_51.0648_ID_5 | FluB-NS-tig00000008
10 | 2320 | 2311 | 1 | 2311 | 2311 | 100.00 | 2395 | 2311 | 96.49 | 100.00 | FluB-PA-NODE_1_length_2395_cov_53.6261_ID_5 | FluB-PA-tig00000002
2 | 2376 | 1 | 2375 | 2375 | 2375 | 100.00 | 2391 | 2375 | 99.33 | 100.00 | FluB-PB1-NODE_1_length_2391_cov_52.3843_ID_72 | FluB-PB1-tig00000004
2 | 2403 | 1 | 2402 | 2402 | 2402 | 99.96 | 2412 | 2402 | 99.59 | 100.00 | FluB-PB2-NODE_1_length_2412_cov_51.4709_ID_5 | FluB-PB2-tig00000009
110 | 1890 | 1781 | 1 | 1781 | 1781 | 100.00 | 1893 | 1782 | 94.08 | 99.94 | H1N1-HA-NODE_1_length_1893_cov_51.9638_ID_5 | H1N1-HA-tig00000026
8 | 1047 | 1 | 1040 | 1040 | 1040 | 100.00 | 1054 | 2071 | 98.67 | 50.22 | H1N1-MP-NODE_1_length_1054_cov_52.2665_ID_5 | H1N1-MP-tig00000004
1 | 1033 | 1033 | 1 | 1033 | 1033 | 100.00 | 1034 | 1463 | 99.90 | 70.61 | H1N1-NA-NODE_1_length_1034_cov_55.4653_ID_36 | H1N1-NA-tig00000022
11 | 1580 | 1586 | 17 | 1570 | 1570 | 100.00 | 1581 | 1586 | 99.30 | 98.99 | H1N1-NP-NODE_1_length_1581_cov_57.4464_ID_5 | H1N1-NP-tig00000000
2 | 891 | 890 | 1 | 890 | 890 | 100.00 | 893 | 908 | 99.66 | 98.02 | H1N1-NS-NODE_1_length_893_cov_69.4517_ID_5 | H1N1-NS-tig00000001
56 | 2298 | 1 | 2240 | 2243 | 2240 | 99.82 | 2307 | 2240 | 97.23 | 100.00 | H1N1-PA-NODE_1_length_2307_cov_53.339_ID_5 | H1N1-PA-tig00000000
3 | 2353 | 1 | 2351 | 2351 | 2351 | 99.96 | 2357 | 2351 | 99.75 | 100.00 | H1N1-PB1-NODE_1_length_2357_cov_53.6274_ID_5 | H1N1-PB1-tig00000000
8 | 2352 | 1 | 2345 | 2345 | 2345 | 100.00 | 2461 | 2345 | 95.29 | 100.00 | H1N1-PB2-NODE_1_length_2461_cov_50.4027_ID_61 | H1N1-PB2-tig00000003
110 | 1875 | 1766 | 1 | 1766 | 1766 | 100.00 | 1883 | 1767 | 93.79 | 99.94 | H3N2-HA-NODE_1_length_1883_cov_55.9431_ID_5 | H3N2-HA-tig00000028
101 | 1142 | 1055 | 15 | 1042 | 1041 | 99.81 | 1145 | 1928 | 91.00 | 53.99 | H3N2-MP-NODE_1_length_1145_cov_60.6287_ID_5 | H3N2-MP-tig00000000
43 | 1513 | 1470 | 1 | 1471 | 1470 | 99.93 | 1520 | 1471 | 96.78 | 99.93 | H3N2-NA-NODE_1_length_1520_cov_51.8306_ID_5 | H3N2-NA-tig00000028
3 | 1581 | 1577 | 1 | 1579 | 1577 | 99.81 | 1583 | 1579 | 99.75 | 99.87 | H3N2-NP-NODE_1_length_1583_cov_51.6312_ID_5 | H3N2-NP-tig00000026
2 | 903 | 1 | 902 | 902 | 902 | 100.00 | 1090 | 1783 | 82.75 | 50.59 | H3N2-NS-NODE_1_length_1090_cov_44.5161_ID_5 | H3N2-NS-tig00000000
2 | 2238 | 1 | 2236 | 2237 | 2236 | 99.96 | 2330 | 2237 | 96.01 | 99.96 | H3N2-PA-NODE_1_length_2330_cov_43.5202_ID_5 | H3N2-PA-tig00000003
4 | 2347 | 1 | 2344 | 2344 | 2344 | 100.00 | 2361 | 2345 | 99.28 | 99.96 | H3N2-PB1-NODE_1_length_2361_cov_58.5027_ID_5 | H3N2-PB1-tig00000007
8 | 2351 | 1 | 2344 | 2344 | 2344 | 100.00 | 2453 | 2345 | 95.56 | 99.96 | H3N2-PB2-NODE_1_length_2453_cov_52.9497_ID_5 | H3N2-PB2-tig00000004

![pacbio vs miseq assemblies](docs/pacbio-vs-miseq-assemblies.png)

These are pretty good. It seems like there's a bit of room for improvement. The figure is ugly but it gets the point across for now. Most of the segments are 100% identical with most of the variants found at the termini. So this says that it is possible to assemble clean(er) single molecule Flu sequencing with the current strategy. However, it seems like it is impossible to do with the MinION data as of now.

In [ ]:
cd /home/alan/projects/MinION-notebook/pacbio-canu/FluB-NA-canu-out

bwa mem  -t 30 -x pacbio flu.unassembled.fasta ../Elodie_sidebar_reads_of_insert.fastq > flu.sam
samtools view -b -o flu.bam flu.sam && samtools sort -o flu.sort.bam flu.bam && samtools index flu.sort.bam

contig | reads mapped
-------|-------------
tig00000000 | 1966
tig00000001 | 28
tig00000002 | 3
tig00000003 | 88
tig00000004 | 103
tig00000005 | 71
tig00000006 | 66
tig00000007 | 18



# Canu MinION Lab Strain Assembly

This will be an assessment on whether cleaner data can be assembled with Canu. If not, and if there's a basal error level, then the effort to assembly may be futile, at least with Canu.

## H1N1

In [ ]:
cd /home/alan/projects/MinION-notebook/minion-canu/lab-strains/h1n1

ln -s ~/projects/MinION-notebook/minion-blast/4-20/flu-4-20.fludb.blastn.xml
ln -s ~/projects/MinION-notebook/raw-data/MinION/4-20/flu-4-20.fastq

seg='PB2'
segNum='1'
segSize='2.3k'
subtype='H1N1'

grep -h -P -B6 'Influenza A.+Segment:'$segNum'\|Subtype:'$subtype *xml | grep 'Iteration_query-def' | \
perl -pe 's/^.+>(.+) FAS.+/$1/g' | grep --no-group-separator -A3 -F -f - flu-4-20.fastq  > $subtype-$seg.fastq

canu maxThreads=30 maxMemory=100 stopOnReadQuality=false -p flu -d $subtype-$seg-canu-out \
genomeSize=$segSize corMhapSensitivity=high corMinCoverage=2 errorRate=0.035 minOverlapLength=1499 \
minReadLength=1500 corMaxEvidenceErate=0.3 -nanopore-raw $subtype-$seg.fastq

seg='PB1'
segNum='2'
segSize='2.3k'

grep -h -P -B6 'Influenza A.+Segment:'$segNum'\|Subtype:'$subtype *xml | grep 'Iteration_query-def' | \
perl -pe 's/^.+>(.+) FAS.+/$1/g' | grep --no-group-separator -A3 -F -f - flu-4-20.fastq  > $subtype-$seg.fastq

canu maxThreads=30 maxMemory=100 stopOnReadQuality=false -p flu -d $subtype-$seg-canu-out \
genomeSize=$segSize corMhapSensitivity=high corMinCoverage=2 errorRate=0.035 minOverlapLength=1499 \
minReadLength=1500 corMaxEvidenceErate=0.3 -nanopore-raw $subtype-$seg.fastq

seg='PA'
segNum='3'
segSize='2.2k'

grep -h -P -B6 'Influenza A.+Segment:'$segNum'\|Subtype:'$subtype *xml | grep 'Iteration_query-def' | \
perl -pe 's/^.+>(.+) FAS.+/$1/g' | grep --no-group-separator -A3 -F -f - flu-4-20.fastq  > $subtype-$seg.fastq

canu maxThreads=30 maxMemory=100 stopOnReadQuality=false -p flu -d $subtype-$seg-canu-out \
genomeSize=$segSize corMhapSensitivity=high corMinCoverage=2 errorRate=0.035 minOverlapLength=1499 \
minReadLength=1500 corMaxEvidenceErate=0.3 -nanopore-raw $subtype-$seg.fastq

seg='HA'
segNum='4'
segSize='1.7k'

grep -h -P -B6 'Influenza A.+Segment:'$segNum'\|Subtype:'$subtype *xml | grep 'Iteration_query-def' | \
perl -pe 's/^.+>(.+) FAS.+/$1/g' | grep --no-group-separator -A3 -F -f - flu-4-20.fastq  > $subtype-$seg.fastq

canu maxThreads=30 maxMemory=100 stopOnReadQuality=false -p flu -d $subtype-$seg-canu-out \
genomeSize=$segSize corMhapSensitivity=high corMinCoverage=2 errorRate=0.035 minOverlapLength=1499 \
minReadLength=1500 corMaxEvidenceErate=0.3 -nanopore-raw $subtype-$seg.fastq

seg='NP'
segNum='5'
segSize='1.5k'

grep -h -P -B6 'Influenza A.+Segment:'$segNum'\|Subtype:'$subtype *xml | grep 'Iteration_query-def' | \
perl -pe 's/^.+>(.+) FAS.+/$1/g' | grep --no-group-separator -A3 -F -f - flu-4-20.fastq  > $subtype-$seg.fastq

canu maxThreads=30 maxMemory=100 stopOnReadQuality=false -p flu -d $subtype-$seg-canu-out \
genomeSize=$segSize corMhapSensitivity=high corMinCoverage=2 errorRate=0.035 minOverlapLength=1499 \
minReadLength=1500 corMaxEvidenceErate=0.3 -nanopore-raw $subtype-$seg.fastq

seg='NA'
segNum='6'
segSize='1.4k'

grep -h -P -B6 'Influenza A.+Segment:'$segNum'\|Subtype:'$subtype *xml | grep 'Iteration_query-def' | \
perl -pe 's/^.+>(.+) FAS.+/$1/g' | grep --no-group-separator -A3 -F -f - flu-4-20.fastq  > $subtype-$seg.fastq

canu maxThreads=30 maxMemory=100 stopOnReadQuality=false -p flu -d $subtype-$seg-canu-out \
genomeSize=$segSize corMhapSensitivity=high corMinCoverage=2 minOverlapLength=499 \
-nanopore-raw $subtype-$seg.fastq

seg='MP'
segNum='7'
segSize='1k'

grep -h -P -B6 'Influenza A.+Segment:'$segNum'\|Subtype:'$subtype *xml | grep 'Iteration_query-def' | \
perl -pe 's/^.+>(.+) FAS.+/$1/g' | grep --no-group-separator -A3 -F -f - flu-4-20.fastq  > $subtype-$seg.fastq

canu maxThreads=30 maxMemory=100 stopOnReadQuality=false -p flu -d $subtype-$seg-canu-out \
genomeSize=$segSize corMhapSensitivity=high corMinCoverage=2 minOverlapLength=499 \
-nanopore-raw $subtype-$seg.fastq

seg='NS'
segNum='8'
segSize='900'

grep -h -P -B6 'Influenza A.+Segment:'$segNum'\|Subtype:'$subtype *xml | grep 'Iteration_query-def' | \
perl -pe 's/^.+>(.+) FAS.+/$1/g' | grep --no-group-separator -A3 -F -f - flu-4-20.fastq  > $subtype-$seg.fastq

canu maxThreads=30 maxMemory=100 stopOnReadQuality=false -p flu -d $subtype-$seg-canu-out \
genomeSize=$segSize minReadLength=500 corMhapSensitivity=high corMinCoverage=2 minOverlapLength=499 \
-nanopore-raw $subtype-$seg.fastq

for i in HA NA NP NS MP PA PB1 PB2; do perl -pe 's/^>(tig\d+)/>'$i'-$1/g' $subtype-$i-canu-out/flu.unassembled.fasta; done \
> $subtype.contigs.fasta

nucmer --maxmatch ~/projects/MinION-notebook/miseq-assembly/lab-strains/$subtype-assembly.fasta $subtype.contigs.fasta
show-coords -lcT <(delta-filter -l 500 -r out.delta ) | perl -pe 's/\t/ | /g'

S1 | E1 | S2 | E2 | LEN 1 | LEN 2 | % IDY | LEN R | LEN Q | COV R | COV Q | TAGS
---|----|----|----|-------|-------|-------|-------|-------|-------|-------|-----
126 | 1889 | 1702 | 4 | 1764 | 1699 | 96.20 | 1893 | 1704 | 93.19 | 99.71 | NODE_1_length_1893_cov_51.9638_ID_5 | HA-tig00000000
18 | 1047 | 1030 | 24 | 1030 | 1007 | 97.67 | 1054 | 1030 | 97.72 | 97.77 | NODE_1_length_1054_cov_52.2665_ID_5 | MP-tig00000007
1 | 1030 | 1038 | 26 | 1030 | 1013 | 98.16 | 1034 | 1486 | 99.61 | 68.17 | NODE_1_length_1034_cov_55.4653_ID_36 | NA-tig00000000
35 | 1570 | 1502 | 1 | 1536 | 1502 | 97.33 | 1581 | 1502 | 97.15 | 100.00 | NODE_1_length_1581_cov_57.4464_ID_5 | NP-tig00000000
1 | 1552 | 25 | 1547 | 1552 | 1523 | 97.94 | 1581 | 1549 | 98.17 | 98.32 | NODE_1_length_1581_cov_57.4464_ID_5 | NP-tig00000021
7 | 882 | 25 | 877 | 876 | 853 | 97.03 | 893 | 877 | 98.10 | 97.26 | NODE_1_length_893_cov_69.4517_ID_5 | NS-tig00000001
1 | 860 | 22 | 868 | 860 | 847 | 98.37 | 893 | 868 | 96.30 | 97.58 | NODE_1_length_893_cov_69.4517_ID_5 | NS-tig00000034
174 | 2298 | 2077 | 1 | 2125 | 2077 | 97.23 | 2307 | 2077 | 92.11 | 100.00 | NODE_1_length_2307_cov_53.339_ID_5 | PA-tig00000006
55 | 2291 | 12 | 2219 | 2237 | 2208 | 98.66 | 2307 | 2219 | 96.97 | 99.50 | NODE_1_length_2307_cov_53.339_ID_5 | PA-tig00000027
1 | 2351 | 8 | 2283 | 2351 | 2276 | 96.77 | 2357 | 2283 | 99.75 | 99.69 | NODE_1_length_2357_cov_53.6274_ID_5 | PB1-tig00000000
93 | 2355 | 2181 | 14 | 2263 | 2168 | 95.67 | 2357 | 2181 | 96.01 | 99.40 | NODE_1_length_2357_cov_53.6274_ID_5 | PB1-tig00000002
28 | 2363 | 2257 | 12 | 2336 | 2246 | 95.68 | 2461 | 2260 | 94.92 | 99.38 | NODE_1_length_2461_cov_50.4027_ID_61 | PB2-tig00000035
9 | 2348 | 1 | 2284 | 2340 | 2284 | 97.44 | 2461 | 2284 | 95.08 | 100.00 | NODE_1_length_2461_cov_50.4027_ID_61 | PB2-tig00000037

## H3N2

In [ ]:
cd /home/alan/projects/MinION-notebook/minion-canu/lab-strains/h3n2

ln -s ~/projects/MinION-notebook/minion-blast/4-20/flu-4-20.fludb.blastn.xml
ln -s ~/projects/MinION-notebook/raw-data/MinION/4-20/flu-4-20.fastq

seg='PB2'
segNum='1'
segSize='2.3k'
subtype='H3N2'

grep -h -P -B6 'Influenza A.+Segment:'$segNum'\|Subtype:'$subtype *xml | grep 'Iteration_query-def' | \
perl -pe 's/^.+>(.+) FAS.+/$1/g' | grep --no-group-separator -A3 -F -f - flu-4-20.fastq  > $subtype-$seg.fastq

canu maxThreads=30 maxMemory=100 stopOnReadQuality=false -p flu -d $subtype-$seg-canu-out \
genomeSize=$segSize corMhapSensitivity=high corMinCoverage=2 errorRate=0.035 minOverlapLength=1499 \
minReadLength=1500 corMaxEvidenceErate=0.3 -nanopore-raw $subtype-$seg.fastq

seg='PB1'
segNum='2'
segSize='2.3k'

grep -h -P -B6 'Influenza A.+Segment:'$segNum'\|Subtype:'$subtype *xml | grep 'Iteration_query-def' | \
perl -pe 's/^.+>(.+) FAS.+/$1/g' | grep --no-group-separator -A3 -F -f - flu-4-20.fastq  > $subtype-$seg.fastq

canu maxThreads=30 maxMemory=100 stopOnReadQuality=false -p flu -d $subtype-$seg-canu-out \
genomeSize=$segSize corMhapSensitivity=high corMinCoverage=2 errorRate=0.035 minOverlapLength=1499 \
minReadLength=1500 corMaxEvidenceErate=0.3 -nanopore-raw $subtype-$seg.fastq

seg='PA'
segNum='3'
segSize='2.2k'

grep -h -P -B6 'Influenza A.+Segment:'$segNum'\|Subtype:'$subtype *xml | grep 'Iteration_query-def' | \
perl -pe 's/^.+>(.+) FAS.+/$1/g' | grep --no-group-separator -A3 -F -f - flu-4-20.fastq  > $subtype-$seg.fastq

canu maxThreads=30 maxMemory=100 stopOnReadQuality=false -p flu -d $subtype-$seg-canu-out \
genomeSize=$segSize corMhapSensitivity=high corMinCoverage=2 errorRate=0.035 minOverlapLength=1499 \
minReadLength=1500 corMaxEvidenceErate=0.3 -nanopore-raw $subtype-$seg.fastq

seg='HA'
segNum='4'
segSize='1.7k'

grep -h -P -B6 'Influenza A.+Segment:'$segNum'\|Subtype:'$subtype *xml | grep 'Iteration_query-def' | \
perl -pe 's/^.+>(.+) FAS.+/$1/g' | grep --no-group-separator -A3 -F -f - flu-4-20.fastq  > $subtype-$seg.fastq

canu maxThreads=30 maxMemory=100 stopOnReadQuality=false -p flu -d $subtype-$seg-canu-out \
genomeSize=$segSize corMhapSensitivity=high corMinCoverage=2 errorRate=0.035 minOverlapLength=1499 \
minReadLength=1500 corMaxEvidenceErate=0.3 -nanopore-raw $subtype-$seg.fastq

seg='NP'
segNum='5'
segSize='1.5k'

grep -h -P -B6 'Influenza A.+Segment:'$segNum'\|Subtype:'$subtype *xml | grep 'Iteration_query-def' | \
perl -pe 's/^.+>(.+) FAS.+/$1/g' | grep --no-group-separator -A3 -F -f - flu-4-20.fastq  > $subtype-$seg.fastq

canu maxThreads=30 maxMemory=100 stopOnReadQuality=false -p flu -d $subtype-$seg-canu-out \
genomeSize=$segSize corMhapSensitivity=high corMinCoverage=2 errorRate=0.035 minOverlapLength=1499 \
minReadLength=1500 corMaxEvidenceErate=0.3 -nanopore-raw $subtype-$seg.fastq

seg='NA'
segNum='6'
segSize='1.4k'

grep -h -P -B6 'Influenza A.+Segment:'$segNum'\|Subtype:'$subtype *xml | grep 'Iteration_query-def' | \
perl -pe 's/^.+>(.+) FAS.+/$1/g' | grep --no-group-separator -A3 -F -f - flu-4-20.fastq  > $subtype-$seg.fastq

canu maxThreads=30 maxMemory=100 stopOnReadQuality=false -p flu -d $subtype-$seg-canu-out \
genomeSize=$segSize corMhapSensitivity=high corMinCoverage=2 minOverlapLength=499 \
-nanopore-raw $subtype-$seg.fastq

seg='MP'
segNum='7'
segSize='1k'

grep -h -P -B6 'Influenza A.+Segment:'$segNum'\|Subtype:'$subtype *xml | grep 'Iteration_query-def' | \
perl -pe 's/^.+>(.+) FAS.+/$1/g' | grep --no-group-separator -A3 -F -f - flu-4-20.fastq  > $subtype-$seg.fastq

canu maxThreads=30 maxMemory=100 stopOnReadQuality=false -p flu -d $subtype-$seg-canu-out \
genomeSize=$segSize corMhapSensitivity=high corMinCoverage=2 minOverlapLength=499 \
-nanopore-raw $subtype-$seg.fastq

seg='NS'
segNum='8'
segSize='900'

grep -h -P -B6 'Influenza A.+Segment:'$segNum'\|Subtype:'$subtype *xml | grep 'Iteration_query-def' | \
perl -pe 's/^.+>(.+) FAS.+/$1/g' | grep --no-group-separator -A3 -F -f - flu-4-20.fastq  > $subtype-$seg.fastq

canu maxThreads=30 maxMemory=100 stopOnReadQuality=false -p flu -d $subtype-$seg-canu-out \
genomeSize=$segSize minReadLength=500 corMhapSensitivity=high corMinCoverage=2 minOverlapLength=499 \
-nanopore-raw $subtype-$seg.fastq

for i in HA NA NP NS MP PA PB1 PB2; do perl -pe 's/^>(tig\d+)/>'$i'-$1/g' $subtype-$i-canu-out/flu.unassembled.fasta; done \
> $subtype.contigs.fasta

nucmer --maxmatch ~/projects/MinION-notebook/miseq-assembly/lab-strains/$subtype-assembly.fasta $subtype.contigs.fasta
show-coords -lcT <(delta-filter -l 500 -r out.delta ) | perl -pe 's/\t/ | /g'

S1 | E1 | S2 | E2 | LEN 1 | LEN 2 | % IDY | LEN R | LEN Q | COV R | COV Q | TAGS
---|----|----|----|-------|-------|-------|-------|-------|-------|-------|-----
91 | 1864 | 1 | 1747 | 1774 | 1747 | 98.20 | 1883 | 1764 | 94.21 | 99.04 | NODE_1_length_1883_cov_55.9431_ID_5 | HA-tig00000000
198 | 1875 | 1654 | 6 | 1678 | 1649 | 97.92 | 1883 | 1655 | 89.11 | 99.64 | NODE_1_length_1883_cov_55.9431_ID_5 | HA-tig00000001
113 | 1144 | 1030 | 23 | 1032 | 1008 | 97.67 | 1145 | 1030 | 90.13 | 97.86 | NODE_1_length_1145_cov_60.6287_ID_5 | MP-tig00000002
36 | 1512 | 1471 | 19 | 1477 | 1453 | 98.38 | 1520 | 1471 | 97.17 | 98.78 | NODE_1_length_1520_cov_51.8306_ID_5 | NA-tig00000000
7 | 1583 | 1546 | 3 | 1577 | 1544 | 97.91 | 1583 | 1546 | 99.62 | 99.87 | NODE_1_length_1583_cov_51.6312_ID_5 | NP-tig00000000
1 | 903 | 23 | 905 | 903 | 883 | 97.79 | 1090 | 923 | 82.84 | 95.67 | NODE_1_length_1090_cov_44.5161_ID_5 | NS-tig00000000
11 | 2190 | 3 | 2127 | 2180 | 2125 | 97.25 | 2330 | 2127 | 93.56 | 99.91 | NODE_1_length_2330_cov_43.5202_ID_5 | PA-tig00000004
14 | 2248 | 2191 | 24 | 2235 | 2168 | 96.91 | 2330 | 2194 | 95.92 | 98.81 | NODE_1_length_2330_cov_43.5202_ID_5 | PA-tig00000020
4 | 2359 | 2284 | 3 | 2356 | 2282 | 96.73 | 2361 | 2295 | 99.79 | 99.43 | NODE_1_length_2361_cov_58.5027_ID_5 | PB1-tig00000000
3 | 2181 | 14 | 2135 | 2179 | 2122 | 97.34 | 2361 | 2137 | 92.29 | 99.30 | NODE_1_length_2361_cov_58.5027_ID_5 | PB1-tig00000002
11 | 2361 | 2287 | 21 | 2351 | 2267 | 96.30 | 2453 | 2287 | 95.84 | 99.13 | NODE_1_length_2453_cov_52.9497_ID_5 | PB2-tig00000000

## FluB 4-20

In [ ]:
cd /home/alan/projects/MinION-notebook/minion-canu/lab-strains/flub-420

ln -s ~/projects/MinION-notebook/minion-blast/4-20/flu-4-20.fludb.blastn.xml
ln -s ~/projects/MinION-notebook/raw-data/MinION/4-20/flu-4-20.fastq

seg='PB2'
segNum='1'
segSize='2.3k'
subtype='FluB'

grep -h -P -B6 'Influenza B.+Segment:'$segNum *xml | grep 'Iteration_query-def' | \
perl -pe 's/^.+>(.+) FAS.+/$1/g' | grep --no-group-separator -A3 -F -f - flu-4-20.fastq  > $subtype-$seg.fastq

canu maxThreads=30 maxMemory=100 stopOnReadQuality=false -p flu -d $subtype-$seg-canu-out \
genomeSize=$segSize corMhapSensitivity=high corMinCoverage=2 errorRate=0.035 minOverlapLength=1499 \
minReadLength=1500 corMaxEvidenceErate=0.3 -nanopore-raw $subtype-$seg.fastq

seg='PB1'
segNum='2'
segSize='2.3k'

grep -h -P -B6 'Influenza B.+Segment:'$segNum *xml | grep 'Iteration_query-def' | \
perl -pe 's/^.+>(.+) FAS.+/$1/g' | grep --no-group-separator -A3 -F -f - flu-4-20.fastq  > $subtype-$seg.fastq

canu maxThreads=30 maxMemory=100 stopOnReadQuality=false -p flu -d $subtype-$seg-canu-out \
genomeSize=$segSize corMhapSensitivity=high corMinCoverage=2 errorRate=0.035 minOverlapLength=1499 \
minReadLength=1500 corMaxEvidenceErate=0.3 -nanopore-raw $subtype-$seg.fastq

seg='PA'
segNum='3'
segSize='2.2k'

grep -h -P -B6 'Influenza B.+Segment:'$segNum *xml | grep 'Iteration_query-def' | \
perl -pe 's/^.+>(.+) FAS.+/$1/g' | grep --no-group-separator -A3 -F -f - flu-4-20.fastq  > $subtype-$seg.fastq

canu maxThreads=30 maxMemory=100 stopOnReadQuality=false -p flu -d $subtype-$seg-canu-out \
genomeSize=$segSize corMhapSensitivity=high corMinCoverage=2 errorRate=0.035 minOverlapLength=1499 \
minReadLength=1500 corMaxEvidenceErate=0.3 -nanopore-raw $subtype-$seg.fastq

seg='HA'
segNum='4'
segSize='1.7k'

grep -h -P -B6 'Influenza B.+Segment:'$segNum *xml | grep 'Iteration_query-def' | \
perl -pe 's/^.+>(.+) FAS.+/$1/g' | grep --no-group-separator -A3 -F -f - flu-4-20.fastq  > $subtype-$seg.fastq

canu maxThreads=30 maxMemory=100 stopOnReadQuality=false -p flu -d $subtype-$seg-canu-out \
genomeSize=$segSize corMhapSensitivity=high corMinCoverage=2 errorRate=0.035 minOverlapLength=1499 \
minReadLength=1500 corMaxEvidenceErate=0.3 -nanopore-raw $subtype-$seg.fastq

seg='NP'
segNum='5'
segSize='1.5k'

grep -h -P -B6 'Influenza B.+Segment:'$segNum *xml | grep 'Iteration_query-def' | \
perl -pe 's/^.+>(.+) FAS.+/$1/g' | grep --no-group-separator -A3 -F -f - flu-4-20.fastq  > $subtype-$seg.fastq

canu maxThreads=30 maxMemory=100 stopOnReadQuality=false -p flu -d $subtype-$seg-canu-out \
genomeSize=$segSize corMhapSensitivity=high corMinCoverage=2 errorRate=0.035 minOverlapLength=1499 \
minReadLength=1500 corMaxEvidenceErate=0.3 -nanopore-raw $subtype-$seg.fastq

seg='NA'
segNum='6'
segSize='1.4k'

grep -h -P -B6 'Influenza B.+Segment:'$segNum *xml | grep 'Iteration_query-def' | \
perl -pe 's/^.+>(.+) FAS.+/$1/g' | grep --no-group-separator -A3 -F -f - flu-4-20.fastq  > $subtype-$seg.fastq

canu maxThreads=30 maxMemory=100 stopOnReadQuality=false -p flu -d $subtype-$seg-canu-out \
genomeSize=$segSize corMhapSensitivity=high corMinCoverage=2 minOverlapLength=499 \
-nanopore-raw $subtype-$seg.fastq

seg='MP'
segNum='7'
segSize='1k'

grep -h -P -B6 'Influenza B.+Segment:'$segNum *xml | grep 'Iteration_query-def' | \
perl -pe 's/^.+>(.+) FAS.+/$1/g' | grep --no-group-separator -A3 -F -f - flu-4-20.fastq  > $subtype-$seg.fastq

canu maxThreads=30 maxMemory=100 stopOnReadQuality=false -p flu -d $subtype-$seg-canu-out \
genomeSize=$segSize corMhapSensitivity=high corMinCoverage=2 minOverlapLength=499 \
-nanopore-raw $subtype-$seg.fastq

seg='NS'
segNum='8'
segSize='900'

grep -h -P -B6 'Influenza B.+Segment:'$segNum *xml | grep 'Iteration_query-def' | \
perl -pe 's/^.+>(.+) FAS.+/$1/g' | grep --no-group-separator -A3 -F -f - flu-4-20.fastq  > $subtype-$seg.fastq

canu maxThreads=30 maxMemory=100 stopOnReadQuality=false -p flu -d $subtype-$seg-canu-out \
genomeSize=$segSize minReadLength=500 corMhapSensitivity=high corMinCoverage=2 minOverlapLength=499 \
-nanopore-raw $subtype-$seg.fastq

for i in HA NA NP NS MP PA PB1 PB2; do perl -pe 's/^>(tig\d+)/>'$i'-$1/g' $subtype-$i-canu-out/flu.unassembled.fasta; done \
> $subtype.contigs.fasta

nucmer --maxmatch ~/projects/MinION-notebook/miseq-assembly/lab-strains/$subtype-assembly.fasta $subtype.contigs.fasta
show-coords -lcT <(delta-filter -l 500 -r out.delta ) | perl -pe 's/\t/ | /g'

S1 | E1 | S2 | E2 | LEN 1 | LEN 2 | % IDY | LEN R | LEN Q | COV R | COV Q | TAGS
---|----|----|----|-------|-------|-------|-------|-------|-------|-------|-----
4 | 1900 | 9 | 1859 | 1897 | 1851 | 97.58 | 1906 | 1869 | 99.53 | 99.04 | NODE_1_length_1906_cov_50.5003_ID_5 | HA-tig00000000
24 | 1905 | 1850 | 15 | 1882 | 1836 | 97.40 | 1906 | 1850 | 98.74 | 99.24 | NODE_1_length_1906_cov_50.5003_ID_5 | HA-tig00000002
19 | 1211 | 1164 | 19 | 1193 | 1146 | 95.89 | 1217 | 1164 | 98.03 | 98.45 | NODE_1_length_1217_cov_56.7908_ID_5 | MP-tig00000025
102 | 1673 | 1565 | 17 | 1572 | 1549 | 98.47 | 1676 | 1583 | 93.79 | 97.85 | NODE_1_length_1676_cov_52.7844_ID_5 | NA-tig00000000
9 | 1724 | 1669 | 14 | 1716 | 1656 | 96.39 | 1988 | 1768 | 86.32 | 93.67 | NODE_1_length_1988_cov_51.8023_ID_3 | NP-tig00000014
1 | 1721 | 1682 | 1 | 1721 | 1682 | 97.56 | 1988 | 1769 | 86.57 | 95.08 | NODE_1_length_1988_cov_51.8023_ID_3 | NP-tig00000019
149 | 1257 | 14 | 1101 | 1109 | 1088 | 98.11 | 1269 | 1101 | 87.39 | 98.82 | NODE_1_length_1269_cov_51.0648_ID_5 | NS-tig00000000
165 | 1263 | 1067 | 14 | 1099 | 1054 | 95.72 | 1269 | 1067 | 86.60 | 98.78 | NODE_1_length_1269_cov_51.0648_ID_5 | NS-tig00000001
1 | 2277 | 5 | 2203 | 2277 | 2199 | 96.40 | 2395 | 2203 | 95.07 | 99.82 | NODE_1_length_2395_cov_53.6261_ID_5 | PA-tig00000001
127 | 2317 | 2104 | 1 | 2191 | 2104 | 95.35 | 2395 | 2104 | 91.48 | 100.00 | NODE_1_length_2395_cov_53.6261_ID_5 | PA-tig00000032
19 | 2384 | 4 | 2265 | 2366 | 2262 | 95.44 | 2412 | 2265 | 98.09 | 99.87 | NODE_1_length_2412_cov_51.4709_ID_5 | PB1-tig00000005
11 | 2335 | 1 | 2243 | 2325 | 2243 | 96.43 | 2412 | 2249 | 96.39 | 99.73 | NODE_1_length_2412_cov_51.4709_ID_5 | PB1-tig00000009
3 | 2387 | 1 | 2318 | 2385 | 2318 | 97.11 | 2391 | 2324 | 99.75 | 99.74 | NODE_1_length_2391_cov_52.3843_ID_72 | PB2-tig00000000

# Conclusion

With these findings we can say that Canu can successfully assemble error prone single molecule sequencing (PacBio) but cannot assemble MinION data, most likely due to the low coverage and high error rate. However, the more coverage the better the assemblies. 

# Nanopolish with Canu

Nanopolish requires an alignment of some sort so not sure if this is a good idea since the reference base bias that I uncovered may have an adverse effect on the data. I will try it with the top blast segments first - this will allow me to see if this can resolve the reference base bias.

In [ ]:
cd /data1/share/scratch/minion-canu/flub-only/nanopolish

ln -s ~/projects/MinION-notebook/clinical-analysis/consensus-comparison/flub/flu-11-9.2d.11-9-top-seg-hits.sort.bam
ln -s ~/projects/MinION-notebook/clinical-analysis/consensus-comparison/flub/flu-11-9.2d.11-9-top-seg-hits.sort.bam.bai
ln -s ~/projects/MinION-notebook/clinical-analysis/consensus-comparison/flub/11-9-top-seg-hits.fasta
ln -s ../flu-11-9.2d.fastq

samtools view -F 4 flu-11-9.2d.11-9-top-seg-hits.sort.bam | cut -f1 | grep --no-group-separator \
-A3 -F -f - flu-11-9.2d.fastq > flu-11-9.2d.11-9-top-seg-hits.flub-only.fastq

perl ~/custom-scripts/fq2fa.pl flu-11-9.2d.11-9-top-seg-hits.flub-only.fastq > flu-11-9.2d.11-9-top-seg-hits.flub-only.fasta

#had to change the makefile to be compatible with the new version of samtools

make -f /opt/bioinformatics-software/minion-software/nanopolish/scripts/consensus.make \
READS=flu-11-9.2d.11-9-top-seg-hits.flub-only.fasta ASSEMBLY=11-9-top-seg-hits.fasta



# Canu Falcon Sense Corrected Reads